In [ ]:
from src import data_utils, data_set, lstm_model, transformer
TOKENIZER = data_set.TOKENIZER

Prepare datasets and data loaders

In [ ]:
clean = data_utils.clean_up(data_utils.load_dataset(
    "data/raw_dataset.txt", cap=100000))

splits = data_utils.split_dataset(clean)

data_sets = []
data_loaders = []
for split_name in splits:
    shuffle = 'test' == split_name
    dset, dloader = data_set.prepare_data(
        splits[split_name], shuffle)
    data_sets.append(dset)
    data_loaders.append(dloader)

Train LSTM model

In [ ]:
lstm = lstm_model.Lstm(TOKENIZER)
lstm_model.train(lstm, n_epochs=3, l_rate=0.002, tokenizer=TOKENIZER,
                 train_loader=data_loaders[0], val_loader=data_loaders[1])

LSTM inference and autoregression

In [ ]:
lstm_model.inference(
    lstm, loader=data_loaders[2], tokenizer=TOKENIZER)

while True:
    user_input = input("Enter sentence, \"quit\" to exit: ")
    if user_input == "quit":
        break

    # Predict last 1/4 of sentence
    num_words = max(1, len(user_input.split()) // 3)

    # Autoregression is done via concatenation of user input and
    # model output
    for i in range(0, num_words):
        clean = data_utils.clean_up([user_input])
        _, dloader = data_set.prepare_data(clean)
        user_input = user_input + " " + \
            lstm_model.inference(lstm, loader=dloader,
                                 tokenizer=TOKENIZER, interactive=True)[-1]
    print(user_input)

DistilGPT2 inference and autoregression

In [ ]:
gpt2 = transformer.DistilGPT2()
gpt2.inference(splits["test"])

while True:
    user_input = input("Enter sentence, \"quit\" to exit: ")
    if user_input == "quit":
        break

    # Predict last 1/4 of sentence
    num_words = max(1, len(user_input.split()) // 3)

    # Autoregression is done via concatenation of user input and
    # model output
    for i in range(0, num_words):
        clean = data_utils.clean_up([user_input])
        user_input = user_input + " " + gpt2.autocomplete(user_input)
    print(user_input)